# Practice using F1 data from Ergast API (compiled by Kaggle)
[F1 World Championship 1950-2023](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [1]:
%pip install pandas sqlalchemy ipython-sql

Note: you may need to restart the kernel to use updated packages.


In [106]:
# Load packages
import pandas as pd
import zipfile
import os
from sqlalchemy import create_engine

# Connect to an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Configure the SQL magic
%load_ext sql
%config SqlMagic.autopandas = True  # Return Pandas DataFrames instead of regular result sets
%config SqlMagic.feedback = False # after running query, do not produce output of "Running query in 'sqlite:///chinook.sqlite'"
%config SqlMagic.displaycon = False # Don't show connection string after executing a query

# Use %sql magic to connect to the SQLite database
%sql sqlite:///:memory:

# Unzip the data
with zipfile.ZipFile('F1.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# Load each CSV into a DataFrame
tables = []
for f in os.listdir('data'):
    if f.endswith('.csv'):
        table_name = f.replace('.csv', '') # take 'f.csv' --> returns 'f'
        file_path = os.path.join('data', f) # takes 'f.csv' --> returns 'data/f.csv'
        # tables[table_name] = pd.read_csv(file_path)  <- create variables that hold each csv as a dataframe
        tables.append(table_name)
        globals()[table_name] = pd.read_csv(file_path)


# Use %sql magic to create temporary tables
for table_name in tables:
    %sql DROP TABLE IF EXISTS {table_name}
    %sql --persist {table_name}

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [107]:
globals()['results']

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26075,26081,1110,817,213,3,19,16,16,16,0.0,44,+1:43.071,5053521,25,15,1:50.994,227.169,1
26076,26082,1110,858,3,2,18,17,17,17,0.0,44,+1:44.476,5054926,37,9,1:50.486,228.213,1
26077,26083,1110,807,210,27,0,18,18,18,0.0,44,+1:50.450,5060900,26,4,1:49.907,229.415,1
26078,26084,1110,832,6,55,4,\N,R,19,0.0,23,\N,\N,9,19,1:53.138,222.864,130


In [108]:
# Query using %sql magic
res = %sql SELECT * FROM results LIMIT 5
display(res)


,index,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [109]:
%sql SELECT * FROM constructor_standings LIMIT 5

,index,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,0,1,18,1,14.0,1,1,1
1,1,2,18,2,8.0,3,3,0
2,2,3,18,3,9.0,2,2,0
3,3,4,18,4,5.0,4,4,0
4,4,5,18,5,2.0,5,5,0


In [110]:
%sql SELECT * FROM constructors LIMIT 5

,index,constructorId,constructorRef,name,nationality,url
0,0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [111]:
%%sql
SELECT cs.position, c.name
FROM constructor_standings cs
LEFT JOIN constructors c on cs.constructorId=c.constructorId
ORDER BY position
LIMIT 5

,position,name
0,1,McLaren
1,1,McLaren
2,1,BMW Sauber
3,1,Ferrari
4,1,Ferrari


In [115]:
%%sql
SELECT pit_stops.raceId, pit_stops.driverId, pit_stops.duration, results.constructorId
FROM pit_stops
LEFT JOIN results ON pit_stops.raceId = results.raceId AND pit_stops.driverId = results.driverId
ORDER BY pit_stops.time
Limit 5

,raceId,driverId,duration,constructorId
0,986,20,32.037,6
1,966,817,22.087,9
2,986,1,29.283,131
3,943,20,28.070,6
4,966,835,23.897,4


In [131]:
%%sql
WITH longest_pit AS (
    SELECT pit_stops.raceId, pit_stops.driverId, pit_stops.duration, pit_stops.stop, pit_stops.lap, results.constructorId
    FROM pit_stops
    LEFT JOIN results ON pit_stops.raceId = results.raceId AND pit_stops.driverId = results.driverId
    ORDER BY -pit_stops.duration
    Limit 10
)

SELECT longest_pit.duration, longest_pit.stop, longest_pit.lap, races.name, races.year, drivers.forename, drivers.surname,  constructors.name
FROM longest_pit
LEFT JOIN drivers ON longest_pit.driverId=drivers.driverId
LEFT JOIN races ON longest_pit.raceId=races.raceId
LEFT JOIN constructors ON longest_pit.constructorId=constructors.constructorId




,duration,stop,lap,name,year,forename,surname,name
0,59.555,5,52,Hungarian Grand Prix,2015,Kimi,Räikkönen,Ferrari
1,59.291,1,1,Belgian Grand Prix,2019,Carlos,Sainz,McLaren
2,58.563,1,1,Chinese Grand Prix,2016,Romain,Grosjean,Haas F1 Team
3,57.601,3,33,Emilia Romagna Grand Prix,2021,Fernando,Alonso,Alpine F1 Team
4,57.151,2,39,Russian Grand Prix,2016,Pascal,Wehrlein,Manor Marussia
5,56.733,5,33,Emilia Romagna Grand Prix,2021,Esteban,Ocon,Alpine F1 Team
6,56.710,1,1,Azerbaijan Grand Prix,2018,Fernando,Alonso,McLaren
7,56.611,2,16,Turkish Grand Prix,2011,Vitantonio,Liuzzi,HRT
8,56.083,4,33,Emilia Romagna Grand Prix,2021,Pierre,Gasly,AlphaTauri
9,55.789,1,24,Australian Grand Prix,2015,Carlos,Sainz,Toro Rosso


,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842
...,...,...,...,...,...,...,...
10084,1110,4,2,29,15:59:01,23.798,23798
10085,1110,830,2,30,16:00:16,23.012,23012
10086,1110,848,3,33,16:07:06,23.529,23529
10087,1110,858,3,34,16:09:09,23.109,23109


In [40]:
races

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00
1097,1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N
1098,1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00
1099,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N


In [41]:
drivers

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
852,854,mick_schumacher,47,MSC,Mick,Schumacher,1999-03-22,German,http://en.wikipedia.org/wiki/Mick_Schumacher
853,855,zhou,24,ZHO,Guanyu,Zhou,1999-05-30,Chinese,http://en.wikipedia.org/wiki/Zhou_Guanyu
854,856,de_vries,21,DEV,Nyck,de Vries,1995-02-06,Dutch,http://en.wikipedia.org/wiki/Nyck_de_Vries
855,857,piastri,81,PIA,Oscar,Piastri,2001-04-06,Australian,http://en.wikipedia.org/wiki/Oscar_Piastri


In [132]:

results



,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26075,26081,1110,817,213,3,19,16,16,16,0.0,44,+1:43.071,5053521,25,15,1:50.994,227.169,1
26076,26082,1110,858,3,2,18,17,17,17,0.0,44,+1:44.476,5054926,37,9,1:50.486,228.213,1
26077,26083,1110,807,210,27,0,18,18,18,0.0,44,+1:50.450,5060900,26,4,1:49.907,229.415,1
26078,26084,1110,832,6,55,4,\N,R,19,0.0,23,\N,\N,9,19,1:53.138,222.864,130
